<a href="https://www.kaggle.com/code/taanieluleksin/modified-standard-configuration-with-lkh-60th?scriptVersionId=253005958" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Install LKH

In [1]:
!wget http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.8.tgz
!tar xvfz LKH-3.0.8.tgz
!cd LKH-3.0.8; make; cp LKH ..

--2025-07-28 19:37:55--  http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.8.tgz
Resolving webhotel4.ruc.dk (webhotel4.ruc.dk)... 130.225.220.230
Connecting to webhotel4.ruc.dk (webhotel4.ruc.dk)|130.225.220.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2318525 (2.2M) [application/x-gzip]
Saving to: ‘LKH-3.0.8.tgz’

LKH-3.0.8.tgz       100%[===================>]   2.21M   581KB/s    in 3.9s    

2025-07-28 19:37:59 (581 KB/s) - ‘LKH-3.0.8.tgz’ saved [2318525/2318525]

LKH-3.0.8/
LKH-3.0.8/pr2392.par
LKH-3.0.8/whizzkids96.atsp
LKH-3.0.8/Makefile
LKH-3.0.8/whizzkids96.par
LKH-3.0.8/pr2392.tsp
LKH-3.0.8/DOC/
LKH-3.0.8/README.txt
LKH-3.0.8/SRC/
LKH-3.0.8/SRC/Penalty_CVRPTW.c
LKH-3.0.8/SRC/RestoreTour.c
LKH-3.0.8/SRC/SolveKMeansSubproblems.c
LKH-3.0.8/SRC/IsCommonEdge.c
LKH-3.0.8/SRC/Penalty_TSPPD.c
LKH-3.0.8/SRC/ReadProblem.c
LKH-3.0.8/SRC/BestKOptMove.c
LKH-3.0.8/SRC/Distance_SPECIAL.c
LKH-3.0.8/SRC/Penalty_TSPDL.c
LKH-3.0.8/SRC/Penalty_PDPTW.c
LKH-3.0.8/

# Functions to operate with configurations

Copied from [Pixel travel map: More directions](https://www.kaggle.com/code/starohub/pixel-travel-map-more-directions)

In [2]:
import numpy as np
import pandas as pd

from functools import reduce
from tqdm import tqdm
import numba as nb

def get_position(config):
    return reduce(lambda p, q: (p[0] + q[0], p[1] + q[1]), config, (0, 0))

def compress_path(path):
    
    if len(path) > 2:
        new_path = []
        max_conf_dist = 1
        r = [[] for _ in range(len(path[0]))]
        for p in path:
            for i, c in enumerate(p):
                if len(r[i]) == 0 or r[i][-1] != c:
                    if c not in r[i]:
                        r[i].append(c)
                    else:
                        r[i] = r[i][:r[i].index(c) + 1]
                    assert r[i][-1] == c
        
        max_conf_dist = max([len(r_) for r_ in r])
        for i in range(max_conf_dist):
            new_conf = []
            for _, r_ in enumerate(r):
                
                if i < len(r_):
                    c_ = r_[i]
                else:
                    c_ = r_[-1]
                new_conf.append(c_)
            new_path.append(new_conf)
        return new_path
    return path

def rotate_link(vector, direction):
    x, y = vector
    if direction == 1:  # counter-clockwise
        if y >= x and y > -x:
            x -= 1
        elif y > x and y <= -x:
            y -= 1
        elif y <= x and y < -x:
            x += 1
        else:
            y += 1
    elif direction == -1:  # clockwise
        if y > x and y >= -x:
            x += 1
        elif y >= x and y < -x:
            y += 1
        elif y < x and y <= -x:
            x -= 1
        else:
            y -= 1
    return (x, y)

def rotate(config, i, direction):
    config = config.copy()
    config[i] = rotate_link(config[i], direction)
    return config

def get_direction(u, v):
    """Returns the sign of the angle from u to v."""
    direction = np.sign(np.cross(u, v))
    if direction == 0 and np.dot(u, v) < 0:
        direction = 1
    return direction

def cartesian_to_array(x, y, shape_):
    m, n = shape_[:2]
    i_ = (n - 1) // 2 - y
    j = (n - 1) // 2 + x
    if i_ < 0 or i_ >= m or j < 0 or j >= n:
        raise ValueError("Coordinates not within given dimensions.")
    return i_, j

#@nb.jit(target_backend='cuda')
def reconfiguration_cost(from_config, to_config):
    diffs = np.abs(np.asarray(from_config) - np.asarray(to_config)).sum(axis=1)
    assert diffs.max() <= 1
    return np.sqrt(diffs.sum())

#@nb.jit(target_backend='cuda')
def color_cost(from_position, to_position, image_, color_scale=3.0):
    return np.abs(image_[to_position] - image_[from_position]).sum() * color_scale


# Total cost of one step: the reconfiguration cost plus the color cost
#@nb.jit(target_backend='cuda')
def step_cost(from_config, to_config, image_):
    pos_from = get_position(from_config)
    pos_to = get_position(to_config)
    from_position = cartesian_to_array(pos_from[0], pos_from[1], image_.shape)
    to_position = cartesian_to_array(pos_to[0], pos_to[1], image_.shape)
    return (reconfiguration_cost(from_config, to_config) +
            color_cost(from_position, to_position, image_))

#@nb.jit(target_backend='cuda')
def total_cost(path, image_):
    cost = 0
    for i_ in range(1, len(path)):
        cost += step_cost(path[i_ - 1], path[i_], image_)
    return cost


def get_path_to_point(config, point):
    """Find a path of configurations to `point` starting at `config`."""
    path = [config]
    # Rotate each link, starting with the largest, until the point can
    # be reached by the remaining links. The last link must reach the
    # point itself.
    for i in range(len(config)):
        link = config[i]
        base = get_position(config[:i])
        relbase = (point[0] - base[0], point[1] - base[1])
        position = get_position(config[:i+1])
        relpos = (point[0] - position[0], point[1] - position[1])
        radius = reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
        # Special case when next-to-last link lands on point.
        if radius == 1 and relpos == (0, 0):
            config = rotate(config, i, 1)
            if get_position(config) == point:  # Thanks @pgeiger
                path.append(config)
                break
            else:
                continue
        while np.max(np.abs(relpos)) > radius:
            direction = get_direction(link, relbase)
            config = rotate(config, i, direction)
            path.append(config)
            link = config[i]
            base = get_position(config[:i])
            relbase = (point[0] - base[0], point[1] - base[1])
            position = get_position(config[:i+1])
            relpos = (point[0] - position[0], point[1] - position[1])
            radius = reduce(lambda r, link: r + max(abs(link[0]), abs(link[1])), config[i+1:], 0)
    assert get_position(path[-1]) == point
    
    path = compress_path(path)
    
    return path

def get_path_to_configuration(from_config, to_config):
    path = [from_config]
    config = from_config.copy()
    while config != to_config:
        for i in range(len(config)):
            config = rotate(config, i, get_direction(config[i], to_config[i]))
        path.append(config)
    assert path[-1] == to_config
    
    path = compress_path(path)
    
    return path

def config_to_string(config):
    return ';'.join([' '.join(map(str, vector)) for vector in config])

In [3]:
df_image = pd.read_csv('/kaggle/input/santa-2022/image.csv')

side = df_image.x.nunique()
radius = df_image.x.max()
image = df_image[['r','g','b']].values.reshape(side,side,-1)

## Create standard configuration function

In [4]:
def standard_config(x, y):
    """Return the preferred configuration (list of eight pairs) for the point (x,y)"""
    x_initial = x
    y_initial = y

    if (x > 0 and y >= 0): # Upper right quadrant
        r = 64
        config = [(r, y-r)] # longest arm points to the right
        x = x - config[0][0]
        while r > 1:
            r = r // 2
            arm_x = np.clip(x, -r, r)
            config.append((arm_x, r)) # arm points up
            x -= arm_x
        arm_x = np.clip(x, -r, r)
        config.append((arm_x, r)) # arm points up
        assert x == arm_x
    
    elif (x >= 0 and y < 0): #Lower right quadrant
        r = 64
        config = [(x-r, -r)] # longest arm points down
        y = y - config[0][1]
        while r > 1:
            r = r // 2
            arm_y = np.clip(y, -r, r)
            config.append((r, arm_y)) # arm points right
            y -= arm_y
        arm_y = np.clip(y, -r, r)
        config.append((r, y)) # arm points right
        assert y == arm_y
    
    elif (x < 0 and y <= 0): # lower left quadrant
        r = 64
        config = [(-r, y+r)] # longest arm points left
        x = x - config[0][0]
        while r > 1:
            r = r // 2
            arm_x = np.clip(x, -r, r)
            config.append((arm_x, -r)) # arm points down
            x -= arm_x
        arm_x = np.clip(x, -r, r)
        config.append((arm_x, -r)) # arm points down
        assert x == arm_x
    
    else: # (x <= 0 and y > 0): #Upper left quadrant
        r = 64
        config = [(x+r, r)] # longest arm points up
        y = y - config[0][1]
        while r > 1:
            r = r // 2
            arm_y = np.clip(y, -r, r)
            config.append((-r, arm_y)) # arm points left
            y -= arm_y
        arm_y = np.clip(y, -r, r)
        config.append((-r, y)) # arm points left
        assert y == arm_y

    #Going from origin and returning to origin
    if ((x_initial == 0) and (y_initial >= 0) and (y_initial < 64)):
        config = [(64,y_initial),(-32,0),(-16,0),(-8,0),(-4,0),(-2,0),(-1,0),(-1,0)]
    if ((x_initial == 1) and (y_initial >= 1) and (y_initial < 64)):
        config = [(64,y_initial-1),(-32,0),(-16,0),(-8,0),(-4,0),(-2,0),(-1,0),(0,1)]

    return config

# Create LKH input

# Calculate symmetric adjacency matrix
Since matrix is symmetric, only upper half is calculated.

In [5]:
#Necessary inputs (taken from minimum spanning tree)
N = 257
NN = N*N
Nmax = 128
def hash_to_xy(i):
    return i//N-128, i%N-128
def xy_to_hash(x,y):
    return (x+128)*N+(y+128)

In [6]:
tsp_mat = np.ones((NN,NN), dtype=np.uint16)*9999

for i in range(NN):
    x, y = hash_to_xy(i)
    for dx in range(-4, 5):
        for dy in range(-4, 5):
            nx, ny = x+dx, y+dy
            if np.abs(dx)+np.abs(dy) > 8:
                continue
            if nx<-Nmax or Nmax<nx or ny<-Nmax or Nmax<ny:
                continue
            j = xy_to_hash(nx, ny)
            if i == j:
                tsp_mat[i,j] = 0
            elif (j > i):
                cost = total_cost(get_path_to_configuration(standard_config(x,y),standard_config(nx,ny)),image)
                tsp_mat[i,j] = np.round(np.clip(cost*1000,0,9999),0).astype(np.uint16)


# Write input file for LKH

In [7]:
#Traveling salesman problem file
with open('santa2022.tsp', "w") as f:
    f.write("NAME: Santa image\n")
    f.write("TYPE: TSP\n")
    f.write("DIMENSION: 66049\n")
    f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
    f.write("EDGE_WEIGHT_FORMAT: UPPER_ROW\n")
    f.write("EDGE_WEIGHT_SECTION\n")

    c = 1
    for line in tsp_mat:
        if c < len(line):
            np.savetxt(f, np.array([line[c:]]), fmt='%i', delimiter = ' ')
            c += 1

In [8]:
with open('santa2022.par', "w") as f:
    f.write("PROBLEM_FILE = santa2022.tsp\n")
    f.write("MOVE_TYPE = 5\n")
    f.write("PATCHING_C = 3\n")
    f.write("PATCHING_A = 2\n")
    f.write("RUNS = 1\n")
    f.write("TIME_LIMIT = 20000\n")
    f.write("OUTPUT_TOUR_FILE = out_santa2022_$.txt\n")
    f.write("TOUR_FILE = best_tour.txt\n")

# Run LKH

In [9]:
import os

In [10]:
os.system('./LKH santa2022.par')

PARAMETER_FILE = santa2022.par
Reading PROBLEM_FILE: "santa2022.tsp" ... done
ASCENT_CANDIDATES = 50
BACKBONE_TRIALS = 0
BACKTRACKING = NO
# BWTSP =
# CANDIDATE_FILE =
CANDIDATE_SET_TYPE = ALPHA
# DISTANCE =
# DEPOT =
# EDGE_FILE =
EXCESS = 1.51403e-05
EXTERNAL_SALESMEN = 0
EXTRA_CANDIDATES = 0 
EXTRA_CANDIDATE_SET_TYPE = QUADRANT
GAIN23 = YES
GAIN_CRITERION = YES
INITIAL_PERIOD = 33024
INITIAL_STEP_SIZE = 1
INITIAL_TOUR_ALGORITHM = WALK
# INITIAL_TOUR_FILE = 
INITIAL_TOUR_FRACTION = 1.000
# INPUT_TOUR_FILE = 
KICK_TYPE = 0
KICKS = 1
# MAX_BREADTH =
MAKESPAN = NO
MAX_CANDIDATES = 5 
MAX_SWAPS = 66049
MAX_TRIALS = 66049
# MERGE_TOUR_FILE =
MOVE_TYPE = 5 
# MTSP_MIN_SIZE =
# MTSP_MAX_SIZE =
# MTSP_OBJECTIVE =
# MTSP_SOLUTION_FILE = 
NONSEQUENTIAL_MOVE_TYPE = 9
# OPTIMUM =
OUTPUT_TOUR_FILE = out_santa2022_$.txt
PATCHING_A = 2 
PATCHING_C = 3 
# PI_FILE = 
POPMUSIC_INITIAL_TOUR = NO
POPMUSIC_MAX_NEIGHBORS = 5
POPMUSIC_SAMPLE_SIZE = 10
POPMUSIC_SOLUTIONS = 50
POPMUSIC_TRIALS = 1
# POPULATIO

0

# Read output

In [11]:
x_start = 0
y_start = 0

tsp_path = np.loadtxt("best_tour.txt",skiprows=6,dtype=str)
tsp_path = tsp_path[:-2]
tsp_path = tsp_path.astype(np.int32)
tsp_path -= 1
origin_index = xy_to_hash(x_start,y_start)
tsp_path = np.roll(tsp_path,-np.where(tsp_path==origin_index)[0][0])

### Put together the result

In [12]:
result = [standard_config(x_start,y_start)]
origin = [(64,0),(-32,0),(-16,0),(-8,0),(-4,0),(-2,0),(-1,0),(-1,0)]

for i in tsp_path[1:]:
    x,y = hash_to_xy(i)
    path1 = get_path_to_configuration(result[-1],standard_config(x,y))
    path2 = get_path_to_configuration(standard_config(x,y),result[-1])
    if(total_cost(path1,image) < total_cost(path2,image)):
        result += path1[1:]
    else:
        #print(total_cost(path1,image) - total_cost(path2,image))
        result += path2[:-1][::-1]
        
result += get_path_to_configuration(result[-1],origin)[1:]
total_cost(result,image)

74346.39201422165

## Remove duplicates if there is any

In [13]:
def find_duplicate_points(path):
    duplicate_points = {}
    for c in path:
        p = get_position(c)
        if p != (0,0):
            duplicate_points[p] = duplicate_points.get(p, 0) + 1
    return duplicate_points
    
def vector_diff_one(path):
    for i in range(len(path) - 1):
        for c0, c1 in zip(path[i], path[i+1]):
            if abs(c0[0] - c1[0]) + abs(c0[1] - c1[1]) > 1:
                return False
    return True

def run_remove(path):
    print("-- run remove --")
    print(f"Current length: {len(path)}")
    duplicate_points = find_duplicate_points(path)

    i = len(path) - 2
    while i >= 0 :
        local_p = path[i:i+3]
        p = get_position(local_p[1])
        new_local_p = compress_path(local_p)
        if vector_diff_one(new_local_p) and duplicate_points.get(p, 0) > 1 and len(new_local_p) < 3:
            path = path[:i+1] + path[i+2:]
            duplicate_points[p] -= 1
        i -= 1
    print(f"New length: {len(path)}")
    return path

In [14]:
result_copy2 = result.copy()
result = run_remove(result)
result = run_remove(result)
total_cost(result,image)

-- run remove --
Current length: 66053
New length: 66052
-- run remove --
Current length: 66052
New length: 66052


74345.70985146683

## Create submission

In [15]:
submission = pd.Series(
    [config_to_string(config) for config in result],
    name="configuration",
)

submission.to_csv('submission.csv', index=False)
submission.head()

0    64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
1    64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 1
2     64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;0 1
3     64 1;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;0 1
4     64 2;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;0 1
Name: configuration, dtype: object